In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import copy
import seaborn as sns
sns.set()

In [49]:
data = pd.read_csv('../Data/fillna_big_df.csv')
data.Datetime = pd.to_datetime(data.Datetime)
data = data[data.Datetime>='2019-10-01'].reset_index(drop=True)
data.head()

,Datetime,temp,desc,templow,icon,baro,wind,wd,hum,rain,...,intensite_greve,etat_barre_ce,q_ce,k_ce,etat_barre_lc,q_lc,k_lc,etat_barre_pv,q_pv,k_pv
0,2019-10-01 00:00:00,2.0,Clear.,1.0,13.0,1026.0,13.0,350.0,88.0,0.0,...,0.0,3.0,767.0,10.40890,3.0,579.0,3.55667,3.0,509.0,6.13445
1,2019-10-01 01:00:00,1.0,Clear.,1.0,13.0,1026.0,13.0,350.0,88.0,0.0,...,0.0,3.0,695.0,8.73556,3.0,386.0,2.21667,3.0,358.0,4.40556
2,2019-10-01 02:00:00,1.0,Clear.,1.0,13.0,1026.0,13.0,350.0,88.0,0.0,...,0.0,3.0,423.0,5.25167,3.0,232.0,1.20111,3.0,201.0,2.48111
3,2019-10-01 03:00:00,1.0,Clear.,1.0,13.0,1026.0,13.0,350.0,88.0,0.0,...,0.0,3.0,370.0,3.79667,3.0,125.0,0.84333,3.0,103.0,1.24111
4,2019-10-01 04:00:00,1.0,Clear.,1.0,13.0,1026.0,13.0,350.0,88.0,0.0,...,0.0,3.0,331.0,3.73000,3.0,113.0,0.79945,3.0,60.0,0.80445


# Feature engineering

In [50]:
df = copy.deepcopy(data)

## Time Feature 

In [51]:
df['hour'] = df['Datetime'].dt.hour
df['dayofweek'] = df['Datetime'].dt.dayofweek
df['quarter'] = df['Datetime'].dt.quarter
df['month'] = df['Datetime'].dt.month
df['year'] = df['Datetime'].dt.year
df['dayofyear'] = df['Datetime'].dt.dayofyear
df['dayofmonth'] = df['Datetime'].dt.day
df['weekofyear'] = df['Datetime'].dt.weekofyear

<ipython-input-51-3b8f22692917>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['Datetime'].dt.weekofyear


In [52]:
df[['hour','dayofweek','quarter','month','year', 'dayofyear','dayofmonth','weekofyear']]

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
0,0,1,4,10,2019,274,1,40
1,1,1,4,10,2019,274,1,40
2,2,1,4,10,2019,274,1,40
3,3,1,4,10,2019,274,1,40
4,4,1,4,10,2019,274,1,40
...,...,...,...,...,...,...,...,...
10243,19,0,4,11,2020,335,30,49
10244,20,0,4,11,2020,335,30,49
10245,21,0,4,11,2020,335,30,49
10246,22,0,4,11,2020,335,30,49


### Hours

In [53]:
df['sin_hour'] = np.sin((2*np.pi/24)*(df.hour))
df['cos_hour'] = np.cos((2*np.pi/24)*(df.hour))
df = df.drop(['hour'], axis=1)

### Dayofweek

In [54]:
df = pd.concat([df, pd.get_dummies(df['dayofweek'], prefix='dayofweek')], axis=1)
df = df.drop(['dayofweek'], axis=1)

### Quarter

In [55]:
df = pd.concat([df, pd.get_dummies(df['quarter'], prefix='quarter')], axis=1)
df = df.drop(['quarter'], axis=1)

### Month

In [56]:
df = pd.concat([df, pd.get_dummies(df['month'], prefix='month')], axis=1)
df = df.drop(['month'], axis=1)

### Dayofyear

In [57]:
df['sin_dayofyear'] = np.sin((2*np.pi/365.2425)*(df.dayofyear))
df['cos_dayofyear'] = np.cos((2*np.pi/365.2425)*(df.dayofyear))
df = df.drop(['dayofyear'], axis=1)

### Weekofyear

In [58]:
df['sin_weekofyear'] = np.sin((2*np.pi/53)*(df.weekofyear))
df['cos_weekofyear'] = np.cos((2*np.pi/53)*(df.weekofyear))
df = df.drop(['weekofyear'], axis=1)

### Dayofmonth

In [59]:
df = pd.concat([df, pd.get_dummies(df['dayofmonth'], prefix='dayofmonth')], axis=1)
df = df.drop(['dayofmonth'], axis=1)

## Weather

### Wind

In [60]:
# Convert to radian
wd_rad = df.wd*np.pi/180
df['Wx'] = df.wind * np.cos(wd_rad)
df['Wy'] = df.wind * np.sin(wd_rad)
df = df.drop(['wind', 'wd'], axis=1)

## Description

In [61]:
values = df.desc
counts = pd.value_counts(values)
mask = values.isin(counts[counts >= 30].index)
df = pd.concat([df, pd.get_dummies(values[mask], prefix='desc')], axis=1)
df = df.drop(['desc'], axis=1)

In [62]:
df.head()

,Datetime,temp,templow,icon,baro,hum,rain,fog,thunder,snow,...,desc_Light rain. Passing clouds.,desc_Low clouds.,desc_More clouds than sun.,desc_Mostly cloudy.,desc_Overcast.,desc_Partly cloudy.,desc_Partly sunny.,desc_Passing clouds.,desc_Scattered clouds.,desc_Sunny.
0,2019-10-01 00:00:00,2.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-10-01 01:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-10-01 02:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-10-01 03:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-10-01 04:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
fill = ['desc_Broken clouds.',
 'desc_Clear.',
 'desc_Drizzle. Fog.',
 'desc_Drizzle. Low clouds.',
 'desc_Drizzle. Mostly cloudy.',
 'desc_Fog.',
 'desc_Ice fog.',
 'desc_Light rain. Fog.',
 'desc_Light rain. Low clouds.',
 'desc_Light rain. Mostly cloudy.',
 'desc_Light rain. Overcast.',
 'desc_Light rain. Partly cloudy.',
 'desc_Light rain. Partly sunny.',
 'desc_Light rain. Passing clouds.',
 'desc_Low clouds.',
 'desc_More clouds than sun.',
 'desc_Mostly cloudy.',
 'desc_Overcast.',
 'desc_Partly cloudy.',
 'desc_Partly sunny.',
 'desc_Passing clouds.',
 'desc_Scattered clouds.',
 'desc_Sunny.']
df[fill] = df[fill].fillna(value=np.float64(0))

In [64]:
df.to_csv('../Notebooks/data.csv', index=False)

# Data Selection

In [17]:
drop = ['etat_barre_ce', 'etat_barre_lc', 'etat_barre_pv', 'Année', 'Mois', 'Jour', 'Heure', 'Jour semaine']
target_ce = ['q_ce', 'k_ce']
target_lc = ['q_lc', 'k_lc']
target_pv = ['q_pv', 'k_pv']
all_ = drop + target_ce + target_lc + target_pv
features = [x for x in df.columns.tolist() if x not in all_]

First we will test without adding the etat barre

In [18]:
df = df.drop(drop, axis=1)

We will create one df per route

In [19]:
df_ce = copy.deepcopy(df[features + target_ce])
df_lc = copy.deepcopy(df[features + target_lc])
df_pv = copy.deepcopy(df[features + target_pv])

In [20]:
df_ce.head()

,Datetime,temp,templow,icon,baro,hum,rain,fog,thunder,snow,...,desc_More clouds than sun.,desc_Mostly cloudy.,desc_Overcast.,desc_Partly cloudy.,desc_Partly sunny.,desc_Passing clouds.,desc_Scattered clouds.,desc_Sunny.,q_ce,k_ce
0,2019-10-01 00:00:00,2.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,767.0,10.40890
1,2019-10-01 01:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,695.0,8.73556
2,2019-10-01 02:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,423.0,5.25167
3,2019-10-01 03:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.0,3.79667
4,2019-10-01 04:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331.0,3.73000


# SARIMAX

In [21]:
from scipy.stats import norm
import statsmodels.api as sm

## Champs Elysées

In [22]:
df_ce = df_ce.set_index('Datetime')

In [23]:
to_keep = ['temp',
 'templow',
 'icon',
 'baro',
 'hum',
 'rain',
 'fog',
 'thunder',
 'snow',
 'Férié',
 'Vacances',
 'confinement_1',
 'confinement_2',
 'confinement_3',
 'curfew',
 'intensite_greve',
 'Wx',
 'Wy',
 'desc_Broken clouds.',
 'desc_Clear.',
 'desc_Drizzle. Fog.',
 'desc_Drizzle. Low clouds.',
 'desc_Drizzle. Mostly cloudy.',
 'desc_Fog.',
 'desc_Ice fog.',
 'desc_Light rain. Fog.',
 'desc_Light rain. Low clouds.',
 'desc_Light rain. Mostly cloudy.',
 'desc_Light rain. Overcast.',
 'desc_Light rain. Partly cloudy.',
 'desc_Light rain. Partly sunny.',
 'desc_Light rain. Passing clouds.',
 'desc_Low clouds.',
 'desc_More clouds than sun.',
 'desc_Mostly cloudy.',
 'desc_Overcast.',
 'desc_Partly cloudy.',
 'desc_Partly sunny.',
 'desc_Passing clouds.',
 'desc_Scattered clouds.',
 'desc_Sunny.']

In [33]:
df_ce

,temp,templow,icon,baro,hum,rain,fog,thunder,snow,Férié,...,desc_More clouds than sun.,desc_Mostly cloudy.,desc_Overcast.,desc_Partly cloudy.,desc_Partly sunny.,desc_Passing clouds.,desc_Scattered clouds.,desc_Sunny.,q_ce,k_ce
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-10-01 00:00:00,2.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,767.0,10.40890
2019-10-01 01:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,695.0,8.73556
2019-10-01 02:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,423.0,5.25167
2019-10-01 03:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.0,3.79667
2019-10-01 04:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331.0,3.73000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-30 19:00:00,3.0,3.0,13.0,1023.0,85.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2020-11-30 20:00:00,3.5,3.0,13.0,1023.0,85.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2020-11-30 21:00:00,4.0,3.0,13.0,1023.0,85.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [38]:
train = df_ce[(df_ce.index < '2020-11-25')]
test = df_ce[(df_ce.index >= '2020-11-25') & (df_ce.index < '2020-11-30')]
exog_x_train = train[to_keep]
y_train = train['q_ce']
exog_x_test = test[to_keep]
y_test = test['q_ce']

In [25]:
exog_train = sm.add_constant(exog_x_train)
# Fit the model
mod = sm.tsa.statespace.SARIMAX(y_train, exog=exog_train, order=(1,0,1))
fit_res = mod.fit(disp=False)
print(fit_res.summary())

/Users/omarsouaidi/opt/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/omarsouaidi/opt/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
/Users/omarsouaidi/opt/miniconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                   q_ce   No. Observations:                10104
Model:               SARIMAX(1, 0, 1)   Log Likelihood              -63003.889
Date:                Mon, 07 Dec 2020   AIC                         126097.778
Time:                        21:16:19   BIC                         126422.709
Sample:                    10-01-2019   HQIC                        126207.709
                         - 11-24-2020                                         
Covariance Type:                  opg                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                             1184.9155    610.510      1.941      0.052     -11.662    2381.493
temp                                15.9596      1.222     13.055

In [39]:
exog_x_test.intensite_greve = exog_x_test.intensite_greve.fillna(value=np.float64(0))

/Users/omarsouaidi/opt/miniconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [40]:
first_predict = exog_x_test.iloc[0].name
exog_test = (sm.add_constant(exog_x_test).loc[first_predict:])
forecast = fit_res.forecast(steps = len(y_test),exog = exog_test)

In [41]:
forecast

2020-11-25 00:00:00    327.229138
2020-11-25 01:00:00    352.227428
2020-11-25 02:00:00    366.067717
2020-11-25 03:00:00    385.113709
2020-11-25 04:00:00    409.718093
                          ...    
2020-11-29 19:00:00    864.234236
2020-11-29 20:00:00    864.234241
2020-11-29 21:00:00    864.234244
2020-11-29 22:00:00    848.274605
2020-11-29 23:00:00    856.254429
Freq: H, Name: predicted_mean, Length: 120, dtype: float64

In [44]:
np.sqrt(np.mean((forecast-y_test)**2))

415.61944432311594

415 less than Xgboost, let's forgive that

https://stackoverflow.com/questions/18016495/get-subset-of-most-frequent-dummy-variables-in-pandas

https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Conv_Full_Exog.ipynb

https://www.tensorflow.org/tutorials/structured_data/time_series#performance_3